In [1]:
from google.cloud import bigquery
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import os
project_id='icu-data-260103' # @param
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id
client = bigquery.Client(location="US")

In [3]:
## Bedside measurements

In [4]:
query = """
    select patientunitstayid, 
    CAST(FLOOR(nursingchartoffset / 10) as INT) AS t,
    avg(case when nursingchartcelltypevallabel = 'Bedside Glucose' then CAST(nursingchartvalue as float64) else null end) as bedside_glucose,
    avg(case when nursingchartcelltypevallabel = 'Non-Invasive BP' then CAST(nursingchartvalue as float64) else null end) as bedside_non_invasive_bp,
    avg(case when nursingchartcelltypevallabel = 'Heart Rate' then CAST(nursingchartvalue as float64) else null end) as bedside_heart_rate,
    avg(case when nursingchartcelltypevallabel = 'Respiratory Rate' then CAST(nursingchartvalue as float64) else null end) as bedside_respiratory_rate,
    avg(case when nursingchartcelltypevallabel = 'O2 Saturation' then CAST(nursingchartvalue as float64) else null end) as bedside_o2_saturation,
    avg(case when nursingchartcelltypevallabel = 'Glasgow coma score' and 
            nursingchartvalue != '' and 
            nursingchartvalue != 'Unable to score due to medication' then CAST(nursingchartvalue as int) else null end) as glasgow_coma_score,
    avg(case when nursingchartcelltypevallabel = 'Invasive BP' then CAST(nursingchartvalue as float64) else null end) as bedside_invasive_bp,
    avg(case when nursingchartcelltypevallabel = 'O2 L/%' then CAST(nursingchartvalue as float64) else null end) as bedside_o2_l_p,
    avg(case when nursingchartcelltypevallabel = 'CVP' then CAST(nursingchartvalue as float64) else null end) as bedside_cvp,
    avg(case when nursingchartcelltypevallabel = 'MAP (mmHg)' then CAST(nursingchartvalue as float64) else null end) as bedside_map
from `icu-data-260103.eicu_crd.nursecharting`
where nursingchartcelltypevallabel in ('Bedside Glucose',
                                        'Non-Invasive BP',
                                        'Heart Rate',
                                        'Respiratory Rate',
                                        'O2 Saturation',
                                        'Glasgow coma score',
                                        'Invasive BP',
                                        'O2 L/%',
                                        'CVP',
                                        'MAP (mmHg)') and
    CAST(FLOOR(nursingchartoffset / 10) as INT) < 10080
group by patientunitstayid, t
order by patientunitstayid, t;
"""
query_job = client.query(
    query,
    # Location must match that of the dataset(s) referenced in the query.
    location="US",
)  # API request - starts the query

df_bedside = query_job.to_dataframe()
df_bedside.columns = df_bedside.columns.str.lower()
print(df_bedside.shape)

(20213118, 12)


In [5]:
df_bedside

,patientunitstayid,t,bedside_glucose,bedside_non_invasive_bp,bedside_heart_rate,bedside_respiratory_rate,bedside_o2_saturation,glasgow_coma_score,bedside_invasive_bp,bedside_o2_l_p,bedside_cvp,bedside_map
0,141168,0,NaN,70.5,140.0,NaN,NaN,NaN,NaN,NaN,NaN,67.0
1,141168,2,NaN,NaN,70.0,NaN,94.0,NaN,NaN,NaN,NaN,75.0
2,141168,3,NaN,82.0,70.0,NaN,94.0,NaN,NaN,NaN,NaN,75.0
3,141168,5,NaN,84.0,140.0,NaN,94.0,NaN,NaN,NaN,NaN,80.0
4,141168,6,NaN,83.5,140.0,NaN,92.0,NaN,NaN,NaN,NaN,80.0
...,...,...,...,...,...,...,...,...,...,...,...,...
20213113,3353263,430,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,NaN
20213114,3353263,483,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,NaN
20213115,3353263,565,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,NaN
20213116,3353263,710,NaN,NaN,NaN,NaN,NaN,7.5,NaN,NaN,NaN,NaN


In [6]:
# forward fill and filter to rows where glucose was measured
df_bedside['glucose_measured'] = ~df_bedside.bedside_glucose.isnull()
ffill_cols = list(df_bedside.columns[~df_bedside.columns.isin(['patientunitstayid','t'])])
df_bedside.loc[:,ffill_cols] = df_bedside.groupby(['patientunitstayid'])[ffill_cols].fillna(method='ffill')  # slow
df_glucose = df_bedside.loc[df_bedside.glucose_measured,:].copy()
del df_bedside

In [10]:
df_glucose.drop(columns=['glucose_measured'])

,bedside_glucose,bedside_non_invasive_bp,bedside_heart_rate,bedside_respiratory_rate,bedside_o2_saturation,glasgow_coma_score,bedside_invasive_bp,bedside_o2_l_p,bedside_cvp,bedside_map,glucose_measured
1721,428.0,101.000000,110.0,19.0,98.0,10.0,NaN,NaN,NaN,NaN,True
1733,444.0,81.000000,88.0,20.0,93.0,10.0,NaN,NaN,NaN,76.0,True
1743,445.0,83.000000,92.0,16.0,98.0,14.0,NaN,NaN,NaN,78.0,True
1747,393.0,84.000000,92.0,11.0,98.0,15.0,NaN,NaN,NaN,78.0,True
1751,342.0,82.500000,96.0,9.0,96.0,15.0,NaN,NaN,NaN,78.0,True
...,...,...,...,...,...,...,...,...,...,...,...
20212969,161.0,76.666667,75.0,21.0,96.0,7.5,76.666667,2.0,NaN,NaN,True
20212975,194.0,84.666667,72.0,22.0,95.0,7.5,76.666667,2.0,NaN,NaN,True
20212976,172.0,82.000000,73.0,20.0,96.0,7.5,76.666667,2.0,NaN,NaN,True
20212981,178.0,77.666667,67.0,21.0,97.0,7.0,76.666667,2.0,NaN,NaN,True


In [ ]:
## vital periodic

In [ ]:
with bedside as (
    select patientunitstayid,
        nursingchartoffset as t,
        nursingchartvalue as bedside_glucose,
    from `icu-data-260103.eicu_crd.nursecharting`
    where nursingchartcelltypevallabel = 'Bedside Glucose'
)
select bedside.patientunitstayid, 
    t,
    observationoffset as t_vital,
    bedside_glucose,
    case when bedside_glucose is not null then 1 else 0 end as glucose_measured,
    temperature as temperature,
    sao2 as sao2,
    heartrate as heartrate,
    respiration as respiration,
    cvp as cvp,
    etco2 as etco2,
    systemicsystolic as systemicsystolic,
    systemicdiastolic as systemicdiastolic,
    systemicmean as systemicmean,
    pasystolic as pasystolic,
    padiastolic as padiastolic,
    pamean as pamean,
    st1 as st1,
    st2 as st2,
    st3 as st3,
    icp as icp,
from `icu-data-260103.eicu_crd.vitalperiodic` period right join bedside on period.patientunitstayid = bedside.patientunitstayid and 
        bedside.t - 15 <= period.observationoffset and bedside.t >= period.observationoffset
order by patientunitstayid, t;

In [7]:
## Insulin

In [8]:
## Nutrition

In [9]:
## Demographics